In [1]:
#import urllib, requests, zipfile, StringIO
#import numpy, scipy
#import matplotlib.pyplot as plt
#import mlpy
import pandas as pd
import numpy as np
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import ExtraTreesRegressor
from time import time
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cross_validation import train_test_split
#from pylab import *
#import seaborn as sns





In [25]:
# import the training data as a "data frame"
train = pd.read_csv("./data/train.csv")
features=train.ix[:,'Feature_1':'Feature_25']
train_=train.dropna()
#test  = pd.read_csv("./data/test.csv")

# heatmap of correlations
feature_corr = train.ix[:,'Feature_1':'Feature_25'].corr()

In [3]:
features.isnull().sum()
null_data = features[features.isnull().any(axis=1)]
null_data.head()
null_data['nans']=null_data.apply(lambda x: sum(x.isnull().values), axis = 1)
null_data.head()

/Users/valeriav/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,nans
0,NaN,NaN,NaN,NaN,8,NaN,75751,0.2254,11,NaN,...,NaN,NaN,-0.925463,2,NaN,-0.489492,NaN,NaN,NaN,13
1,NaN,NaN,NaN,NaN,3,0.388896,17369,0.0166,13,NaN,...,-0.949891,0.917958,-0.897067,9,0.886205,-0.151805,1.239856,0.953424,-0.709462,5
2,NaN,-0.696727,0.739591,-0.167928,9,0.471947,8277,0.3650,9,5,...,-1.132426,1.799813,-1.603488,3,NaN,0.389061,1.728096,-1.798090,-1.019370,2
3,NaN,-0.694350,1.568248,0.479073,5,0.120653,22508,0.2654,13,5,...,-1.131213,1.565036,NaN,7,1.148738,-2.440799,1.551425,-1.788725,NaN,4
5,NaN,NaN,-0.680515,NaN,1,0.227034,24099,0.2064,8,NaN,...,-0.587538,0.230375,-0.897600,10,0.111960,1.670980,0.364626,1.039643,-0.404685,4


In [4]:
#features['Feature_1'].dropna()

In [26]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 929 entries, 4 to 39933
Columns: 211 entries, Id to Weight_Daily
dtypes: float64(209), int64(2)
memory usage: 1.5 MB


In [29]:
# split the data into X and Y
# note that Y is more than one variable
# ignore the last two variables, 'Weight_Intraday' and 'Weight_Daily'. We dont know what to do with them yet.
# We use them only in the scoring function.
X = train_.ix[:,'Feature_1':'Ret_120']
Y = train_.ix[:,'Ret_121':'Ret_PlusTwo']
#print X, Y

In [7]:
# Create an Extremely-Randomized-Trees regressor
extra = ExtraTreesRegressor(n_estimators=100, criterion='mse', max_depth=None, min_samples_split=1, bootstrap=False, n_jobs=-1)
t0=time()
extra = extra.fit(X,Y)
t1=time()
print "time elapsed %.2f seconds" % (t1-t0)
fi=pd.DataFrame(extra.feature_importances_,index=X.columns.values,columns=['importance'])
# fi.head(10)

time elapsed 2.97 seconds


In [8]:

#fi=zip(X.columns.values, extra.feature_importances_)
#type(extra.feature_importances_)
#fi.sort_values(by='importance', ascending=False,inplace=True)
#fi.plot(kind='bar')

In [9]:
'''
this will be the function that replaces missing data by KNN'''
def knn_imputation(df):
    df['nans']=df.apply(lambda x: sum(x.isnull().values), axis = 1)
    df.sort_values(by='nans', ascending=True, inplace=True)
    all_data=df[df['nans']==0]
    #null_data = df[df.isnull().any(axis=1)]
    null_data=df[df['nans']!=0]
    #replace missing values for rows with one variable by knn
    
    

In [187]:
'''
replacing NaNs by KNN. There are many missing values in different columns and rows. Start with a row, let's say row,
with one missing value in column col; then take the Y to be df[0:row-1,col] and X df[0:row-1, all columns except col].
run KNN for X, Y. predict df[row,col]. fill it in. add to the complete data.'''

# null_data = features[features.isnull().any(axis=1)]
# null_data.head()
# null_data['nans']=null_data.apply(lambda x: sum(x.isnull().values), axis = 1)
df=train.ix[:,'Feature_1':'Feature_25'] #keep only the features in df
#df['nans']=df.apply(lambda x: sum(x.isnull().values), axis = 1) #create a column that counts number of NaNs by row
df['nans']=df.apply(lambda x: len(x)-x.count(), axis = 1) #create a column that counts number of NaNs by row
df.sort_values(by='nans', ascending=True, inplace=True)#sort by nr of NaNs. complete cases will be at the top
df=df.reset_index(drop=True) #reset the index after sorting matrix
clean_data=df[df['nans']==0] #not sure if I'll use this
#null_data = df[df.isnull().any(axis=1)]
#null_data=df[df['nans']!=0]
rows_complete=len(df[df.nans==0])
i=0
print df.iloc[rows_complete+1:rows_complete+6,:]
#replace missing values for rows with one variable by knn, have to do it one by one
for row in range(rows_complete,len(df)): #each row in the set of rows that only have one missing value
        if i<10:
            '''code below only works for one missing value currently; should loop through all missing values'''
            col_nan=np.where(df.iloc[row,:].isnull())[0] #gives me the column with NaN for each row --- there's only one to begin with
            print ("working on row ", row, " NaN in column ",df.columns[col_nan])
            data=df.iloc[:row,:]
            data=data.drop(df.columns[col_nan],1)#pick the data frame with complete cases, dropping the column where row has the missing value
            X_na=data.tail(1)
            X=data.iloc[:-1,:]
            Y=df.iloc[:row-1,col_nan]
            myknn = KNeighborsRegressor(n_neighbors=3,weights='distance') 
            myknn.fit(X, Y) #fit KNN to the complete data
            df.iloc[row,col_nan]=myknn.predict(X_na)[0][0] #predict the missing value
            print df.iloc[rows_complete:rows_complete+6,:] 
        i=i+1

      Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6  \
1294        NaN  -0.216722  -0.366769  -0.311828         10  -0.251787   
1295        NaN  -0.726884   0.332485  -0.388388          9   0.294279   
1296        NaN  -1.070952   0.250769   0.995944          2   0.051174   
1297          2  -0.671309   2.208364   1.956778          6   1.720166   
1298        NaN  -0.580954   0.566658   0.369882          8  -0.091067   

      Feature_7  Feature_8  Feature_9  Feature_10  ...   Feature_17  \
1294      19660     0.3318         11           5  ...    -0.150502   
1295      61543     0.3318         14           5  ...    -0.728306   
1296      40712     0.3451          7           5  ...    -0.635299   
1297      73026     0.3451         12           5  ...    -1.991323   
1298      62512     0.3318          8           4  ...    -0.712756   

      Feature_18  Feature_19  Feature_20  Feature_21  Feature_22  Feature_23  \
1294    0.035194   -0.773011           3   -0.38

In [178]:
print data.iloc[:-1,:].shape
print myknn.predict(X_na).ravel()[0]

(1292, 25)
5.40821130474


In [133]:
x=[[1,2,3],[np.NaN,4,np.NaN]]
df=pd.DataFrame(x,columns=['A','B','C'])
print df
n=np.where(df.iloc[1,:].isnull())
print (n)
print (n[0])

    A  B   C
0   1  2   3
1 NaN  4 NaN
(array([0, 2]),)
[0 2]


In [21]:
print len(df[df.nans==5])
print df.nans[39000]
# n=df.columns[df.isnull().any()].tolist()
# v=df.apply(lambda x: )

#df.iloc[:,np.where(df.iloc[1,:].isnull())[0]]
#print df.columns[df.loc[1,:].isnull()]

1365
5


In [61]:
df.columns[df.loc[39093,:].isnull().any()]

'Feature_2'

In [ ]:
sum(df.loc[1,].isnull())
df.loc[1294,]

In [ ]:
len(df.loc[1,])-df.loc[1,].count()

In [130]:
X=df.loc[:row-1,:]
X=X.drop('Feature_1',axis=1)
X.head()

,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,...,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,nans
0,2.206181,0.967546,0.991364,5,0.276959,99861,0.3451,13,1,-1.128764,...,-1.304266,1.577751,-1.611225,5,0.718281,-2.383161,1.274457,-1.319995,-0.857627,0
1,-0.771461,-0.691794,-0.124020,8,-0.525301,49229,0.2109,7,1,0.481658,...,0.438462,-0.799755,0.203649,2,-1.091945,1.477251,-3.920030,-0.044783,2.439240,0
2,-0.829600,0.849521,1.147628,8,0.116856,52425,0.0098,12,5,-0.938268,...,-1.167142,1.773071,-1.894226,2,0.989519,-1.711147,1.590884,-2.520518,-0.867687,0
3,-2.429350,1.783365,1.644329,7,1.970243,7663,0.3650,10,1,-1.971941,...,-1.680767,2.907570,-1.560428,7,2.267714,0.162972,1.876385,-2.728578,3.573777,0
4,1.433687,0.345290,0.181574,7,0.296787,73407,0.3650,9,5,-0.099282,...,-0.480768,0.616429,-0.810760,2,0.345049,0.312556,-4.709789,-1.179404,-0.733228,0


In [151]:
print col_nan
print np.where(df.iloc[row,:].isnull())
print row
print df.loc[row,:]
print df.iloc[1293,0]
print myknn.predict(X_na)

[0]
(array([0]),)
1293
Feature_1            NaN
Feature_2      -0.394105
Feature_3      -1.058764
Feature_4      -0.988599
Feature_5       8.000000
Feature_6      -0.550167
Feature_7     955.000000
Feature_8       0.365000
Feature_9       6.000000
Feature_10      5.000000
Feature_11      1.155393
Feature_12      0.240000
Feature_13      2.000000
Feature_14      1.047233
Feature_15      2.134526
Feature_16      1.000000
Feature_17      0.543516
Feature_18     -0.649243
Feature_19      0.079692
Feature_20      2.000000
Feature_21     -1.139952
Feature_22      1.445777
Feature_23     -0.351408
Feature_24      0.403906
Feature_25      2.036522
nans            1.000000
Name: 1293, dtype: float64
nan
[[ 9.       ]
 [ 8.       ]
 [ 4.       ]
 ..., 
 [ 1.       ]
 [ 4.       ]
 [ 5.4082113]]
